
# Criador Interativo de Plano de Corrida (Jack Daniels)

Notebook pensado para mapear o perfil do atleta de forma profunda, gerar zonas e simular um plano com o código do repositório. Use como vitrine do sistema ou como guia para preencher um novo perfil.



## Como funciona
1. Garante que o diretório atual aponta para a raiz do repositório (ou para a pasta clonada).
2. Importa os módulos centrais (`user_profile`, `training_zones`, `plan_generator`).
3. Coleta informações-chave inspiradas no checklist Jack Daniels (objetivo, fisiologia, histórico, disponibilidade, logística, preferências, lesões).
4. Calcula VDOT/zones e gera um plano-simulação com `PlanGenerator` para validar se os dados fazem sentido.
5. Exporta o perfil e o plano em JSON para reaproveitar depois.


In [ ]:

from pathlib import Path
import os
import sys

REPO_NAME = 'DecisionMaking'
REPO_MARKERS = ('plan_generator.py', 'user_profile.py', 'training_zones.py')

def find_repo_root(start: Path) -> Path:
    for path in [start] + list(start.parents):
        if all((path / marker).exists() for marker in REPO_MARKERS):
            return path
        if (path / REPO_NAME).exists():
            nested = path / REPO_NAME
            if all((nested / marker).exists() for marker in REPO_MARKERS):
                return nested
    return start

starting_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
repo_root = find_repo_root(starting_dir)

if repo_root == starting_dir and not all((repo_root / marker).exists() for marker in REPO_MARKERS):
    raise FileNotFoundError('Não foi possível localizar o repositório DecisionMaking. Clone ou monte o diretório antes de rodar este notebook.')

os.chdir(repo_root)
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print(f'📁 Diretório de trabalho fixado em: {repo_root}')
print('📄 Principais módulos disponíveis:')
for name in sorted(Path(repo_root).glob('*.py')):
    print(f' - {name.name}')


In [ ]:

from datetime import date
import json
from typing import List, Dict

import ipywidgets as widgets
from IPython.display import display, clear_output

from user_profile import UserProfile, RaceGoal
from training_zones import TrainingZones, RaceTime
from plan_generator import PlanGenerator

print('Bibliotecas carregadas com sucesso e diretório conectado ao repositório.')


## Questionário guiado
Preencha as abas abaixo. Campos essenciais estão marcados, mas sinta-se livre para detalhar ao máximo para gerar planos mais fiéis.


In [ ]:

# Utilidades

def distance_to_km(label: str) -> float:
    mapping = {'5K': 5.0, '10K': 10.0, 'Half Marathon': 21.0975, 'Marathon': 42.195}
    return mapping.get(label, None)

days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# --- Seção 1: Objetivo e contexto ---
objective_widgets = {
    'athlete_name': widgets.Text(description='Nome do atleta'),
    'name': widgets.Text(description='Nome da prova', placeholder='Ex.: Maratona de SP'),
    'location': widgets.Text(description='Cidade/País', placeholder='Local da prova'),
    'distance': widgets.Dropdown(options=['5K', '10K', 'Half Marathon', 'Marathon'], description='Distância'),
    'date': widgets.DatePicker(description='Data'),
    'target_time': widgets.Text(description='Meta (HH:MM:SS)', placeholder='03:45:00'),
    'personal_best': widgets.Text(description='PB atual'),
    'test_races': widgets.Textarea(description='Provas teste', placeholder='10K|2024-08-20|Prova A|00:45:00'),
    'secondary_objectives': widgets.Textarea(description='Objetivos sec.', placeholder='Performance\nSaúde geral'),
    'motivation': widgets.Text(description='Motivação', placeholder='Performance / Saúde / Retomada'),
    'logistics': widgets.Textarea(description='Restrições', placeholder='Viagens, clima, terreno...'),
}
objective_box = widgets.VBox(list(objective_widgets.values()))

# --- Seção 2: Fisiologia ---
physiology_widgets = {
    'recent_distance': widgets.Dropdown(options=['5K', '10K', 'Half Marathon', 'Marathon'], description='Prova recente'),
    'recent_time': widgets.Text(description='Tempo recente', placeholder='00:45:30'),
    'hr_resting': widgets.IntText(description='FC repouso', value=0),
    'hr_max': widgets.IntText(description='FC máx', value=0),
    'age': widgets.IntText(description='Idade'),
    'weight_kg': widgets.FloatText(description='Peso (kg)'),
    'height_cm': widgets.FloatText(description='Altura (cm)'),
    'sex': widgets.Dropdown(options=['', 'M', 'F'], description='Sexo'),
}
physiology_box = widgets.VBox(list(physiology_widgets.values()))

# --- Seção 3: Histórico e consistência ---
history_widgets = {
    'years_running': widgets.FloatText(description='Anos correndo', value=0.0),
    'current_weekly_km': widgets.FloatText(description='Volume atual (km/sem)'),
    'average_weekly_km': widgets.FloatText(description='Média últimas 8-12 sem'),
    'recent_peak_weekly_km': widgets.FloatText(description='Pico recente (km)'),
    'initial_weekly_km': widgets.FloatText(description='Volume inicial desejado', value=0.0),
    'consistent_days_per_week': widgets.IntSlider(description='Dias mantidos/sem', value=3, min=1, max=7),
    'tolerated_workouts': widgets.Textarea(description='Treinos tolerados', placeholder='Tempo run, intervalos, progressivo...'),
    'adherence_score': widgets.FloatSlider(description='Aderência prévia (%)', value=80, min=0, max=100),
    'experience_level': widgets.Dropdown(options=['beginner', 'intermediate', 'advanced'], description='Nível'),
}
history_box = widgets.VBox(list(history_widgets.values()))

# --- Seção 4: Disponibilidade ---
availability_widgets = {
    'days_per_week': widgets.IntSlider(description='Dias/semana alvo', value=4, min=1, max=7),
    'hours_per_day': widgets.FloatSlider(description='Tempo/sessão (h)', value=1.0, min=0.5, max=3.0, step=0.25),
    'preferred_time': widgets.Dropdown(options=['', 'morning', 'afternoon', 'evening'], description='Horário'),
    'preferred_days': widgets.SelectMultiple(options=days_of_week, description='Dias preferidos'),
    'preferred_location': widgets.SelectMultiple(options=['track', 'road', 'trail', 'treadmill'], description='Locais'),
    'weekly_schedule': widgets.Textarea(description='Grade semanal', placeholder='Seg: 06:00-07:00 | max=60 | terreno=pista'),
    'stressful_blocks': widgets.Textarea(description='Blocos críticos', placeholder='Monday: morning\nThursday: night'),
    'long_run_preference_days': widgets.SelectMultiple(options=days_of_week, description='Dias p/ longão'),
    'use_alternating_weeks': widgets.Checkbox(description='Alternar semanas (A/B)', value=False),
    'alternate_stressful_blocks': widgets.Textarea(description='Blocos críticos semana B'),
    'alternate_long_run_days': widgets.SelectMultiple(options=days_of_week, description='Longão semana B'),
    'logistics': widgets.Textarea(description='Infra/acessos', placeholder='Pista às terças, academia...'),
    'available_equipment': widgets.Textarea(description='Equipamentos', placeholder='Relógio, pista, esteira...'),
    'default_warmup_minutes': widgets.IntText(description='Aquecimento (min)', value=10),
    'default_cooldown_minutes': widgets.IntText(description='Desaquecimento (min)', value=10),
    'commute_minutes': widgets.IntText(description='Deslocamento (min)', value=0),
}
availability_box = widgets.VBox(list(availability_widgets.values()))

# --- Seção 5: Lesões e prevenção ---
injury_widgets = {
    'previous_injuries': widgets.Textarea(description='Lesões prévias'),
    'current_injuries': widgets.Textarea(description='Lesões atuais'),
    'injury_triggers': widgets.Textarea(description='Gatilhos'),
    'red_zones': widgets.Textarea(description='Restrições'),
    'strength_routines': widgets.Textarea(description='Rotina de força'),
    'impact_limitations': widgets.Textarea(description='Limites de impacto'),
}
injury_box = widgets.VBox(list(injury_widgets.values()))

# --- Seção 6: Preferências e psicologia ---
psych_widgets = {
    'rpe_tolerance': widgets.IntSlider(description='RPE típico (1-10)', value=6, min=1, max=10),
    'long_session_tolerance': widgets.Dropdown(options=['baixa', 'moderada', 'alta'], description='Tolerância longos'),
    'variety_preference': widgets.Dropdown(options=['prefere repetição', 'equilíbrio', 'prefere variedade'], description='Variedade'),
    'social_training': widgets.Textarea(description='Treino social', placeholder='grupos, parceiros...'),
    'routine_vs_fun_balance': widgets.Dropdown(options=['rotina', 'equilibrado', 'diversão'], description='Estilo'),
    'feedback_required': widgets.Checkbox(description='Solicitar feedback frequente', value=True),
}
psych_box = widgets.VBox(list(psych_widgets.values()))

# --- Seção 7: Sessões e zonas ---
session_widgets = {
    'intervals': widgets.Checkbox(description='Intervalos', value=True),
    'tempo': widgets.Checkbox(description='Tempo run', value=True),
    'long_run': widgets.Checkbox(description='Longão', value=True),
    'cross_training': widgets.Checkbox(description='Cross-training', value=False),
}
zone_mix_widgets = {
    'easy': widgets.FloatSlider(description='Easy (%)', value=55, min=0, max=100),
    'tempo': widgets.FloatSlider(description='Tempo (%)', value=25, min=0, max=100),
    'interval': widgets.FloatSlider(description='Interval (%)', value=20, min=0, max=100),
}
session_box = widgets.VBox(list(session_widgets.values()) + list(zone_mix_widgets.values()))

# --- Seção 8: Clima e terreno ---
climate_widgets = {
    'race_weather': widgets.Text(description='Clima da prova', placeholder='Quente/úmido/frio'),
    'race_elevation': widgets.Text(description='Altimetria prova', placeholder='plano/ondulado'),
    'training_terrain': widgets.Text(description='Terreno de treino'),
}
climate_box = widgets.VBox(list(climate_widgets.values()))

# Acordeão com todas as seções
accordion = widgets.Accordion(children=[
    objective_box,
    physiology_box,
    history_box,
    availability_box,
    injury_box,
    psych_box,
    session_box,
    climate_box,
])
for i, title in enumerate([
    '1) Objetivo e contexto',
    '2) Fisiologia e VDOT',
    '3) Histórico e consistência',
    '4) Disponibilidade e logística',
    '5) Lesões e prevenção',
    '6) Preferências e psicologia',
    '7) Sessões e mistura de zonas',
    '8) Clima/terreno da prova',
]):
    accordion.set_title(i, title)

display(accordion)



## Gerar perfil + plano de exemplo
Clique no botão para consolidar as respostas, estimar VDOT com Jack Daniels (se houver prova recente) e gerar um plano exemplo com `PlanGenerator`. Nada é enviado para APIs externas; tudo roda localmente para simular a experiência completa.


In [ ]:

output = widgets.Output()

save_button = widgets.Button(description='Gerar perfil e plano de exemplo', button_style='success', icon='check')


def parse_multiline(text: str) -> List[str]:
    return [item.strip() for item in text.split('
') if item.strip()]


def parse_race_list(text: str) -> List[RaceGoal]:
    races = []
    for line in parse_multiline(text):
        parts = [p.strip() for p in line.split('|')]
        if len(parts) >= 2:
            distance, date_str = parts[0], parts[1]
            name = parts[2] if len(parts) > 2 else ''
            target = parts[3] if len(parts) > 3 else None
            location = parts[4] if len(parts) > 4 else ''
            try:
                race_date = date.fromisoformat(date_str)
            except ValueError:
                continue
            races.append(RaceGoal(distance=distance, date=race_date, name=name, location=location, target_time=target, is_main_goal=False))
    return races


def parse_schedule_text(text: str) -> Dict[str, List[Dict[str, object]]]:
    schedule = {}
    for line in parse_multiline(text):
        if ':' not in line:
            continue
        day_part, info = line.split(':', 1)
        day = day_part.strip().capitalize()
        block = {}
        if '|' in info:
            time_part, *extras = [p.strip() for p in info.split('|')]
        else:
            time_part, extras = info.strip(), []
        if '-' in time_part:
            start, end = [t.strip() for t in time_part.split('-', 1)]
            block['start'] = start
            block['end'] = end
        for extra in extras:
            if extra.lower().startswith('max='):
                try:
                    block['max_minutes'] = int(extra.split('=', 1)[1])
                except ValueError:
                    pass
            if 'terreno=' in extra or 'superficie=' in extra or 'superfície=' in extra:
                surfaces = extra.split('=', 1)[1]
                block['surfaces'] = [s.strip() for s in surfaces.split(',') if s.strip()]
        schedule.setdefault(day, []).append(block)
    return schedule


def parse_day_periods(text: str) -> Dict[str, List[str]]:
    mapping = {}
    for line in parse_multiline(text):
        if ':' in line:
            day, periods = line.split(':', 1)
            mapping[day.strip().capitalize()] = [p.strip() for p in periods.split(',') if p.strip()]
        else:
            mapping[line.strip().capitalize()] = []
    return mapping


def normalize_zone_mix(values: Dict[str, float]) -> Dict[str, float]:
    total = sum(values.values()) or 1.0
    return {k: round(v / total, 2) for k, v in values.items()}


def build_profile(_: widgets.Button):
    with output:
        clear_output()
        # Main race goal
        main_goal = RaceGoal(
            distance=objective_widgets['distance'].value,
            date=objective_widgets['date'].value or date.today(),
            name=objective_widgets['name'].value,
            location=objective_widgets['location'].value,
            is_main_goal=True,
            target_time=objective_widgets['target_time'].value or None,
        )

        schedule = parse_schedule_text(availability_widgets['weekly_schedule'].value)
        zone_mix_raw = {
            'easy': zone_mix_widgets['easy'].value,
            'tempo': zone_mix_widgets['tempo'].value,
            'interval': zone_mix_widgets['interval'].value,
        }

        profile = UserProfile(
            name=objective_widgets['athlete_name'].value,
            age=physiology_widgets['age'].value or 0,
            weight_kg=physiology_widgets['weight_kg'].value or 0.0,
            height_cm=physiology_widgets['height_cm'].value or 0.0,
            gender=physiology_widgets['sex'].value,
            years_running=history_widgets['years_running'].value,
            current_weekly_km=history_widgets['current_weekly_km'].value,
            average_weekly_km=history_widgets['average_weekly_km'].value,
            recent_peak_weekly_km=history_widgets['recent_peak_weekly_km'].value,
            initial_weekly_km=history_widgets['initial_weekly_km'].value or None,
            consistent_days_per_week=history_widgets['consistent_days_per_week'].value,
            tolerated_workouts=parse_multiline(history_widgets['tolerated_workouts'].value),
            adherence_score=history_widgets['adherence_score'].value,
            experience_level=history_widgets['experience_level'].value,
            main_race=main_goal,
            test_races=parse_race_list(objective_widgets['test_races'].value),
            secondary_objectives=parse_multiline(objective_widgets['secondary_objectives'].value),
            days_per_week=availability_widgets['days_per_week'].value,
            hours_per_day=availability_widgets['hours_per_day'].value,
            preferred_time=availability_widgets['preferred_time'].value,
            preferred_location=list(availability_widgets['preferred_location'].value),
            preferred_days=list(availability_widgets['preferred_days'].value),
            stressful_blocks=parse_day_periods(availability_widgets['stressful_blocks'].value),
            long_run_preference_days=list(availability_widgets['long_run_preference_days'].value),
            use_alternating_weeks=availability_widgets['use_alternating_weeks'].value,
            alternate_stressful_blocks=parse_day_periods(availability_widgets['alternate_stressful_blocks'].value),
            alternate_long_run_days=list(availability_widgets['alternate_long_run_days'].value),
            weekly_schedule=schedule,
            default_warmup_minutes=availability_widgets['default_warmup_minutes'].value,
            default_cooldown_minutes=availability_widgets['default_cooldown_minutes'].value,
            commute_minutes=availability_widgets['commute_minutes'].value,
            typical_key_workout_rpe=psych_widgets['rpe_tolerance'].value,
            long_session_tolerance=psych_widgets['long_session_tolerance'].value,
            variety_preference=psych_widgets['variety_preference'].value,
            social_training_options=parse_multiline(psych_widgets['social_training'].value),
            routine_vs_fun_balance=psych_widgets['routine_vs_fun_balance'].value,
            recent_race_times={},
            zones_calculation_method='jack_daniels',
            zone_mix_preference=normalize_zone_mix(zone_mix_raw),
            hr_resting=physiology_widgets['hr_resting'].value or None,
            hr_max=physiology_widgets['hr_max'].value or None,
            session_preferences={k: widget.value for k, widget in session_widgets.items()},
            previous_injuries=parse_multiline(injury_widgets['previous_injuries'].value),
            current_injuries=parse_multiline(injury_widgets['current_injuries'].value),
            injury_triggers=parse_multiline(injury_widgets['injury_triggers'].value),
            red_zones=parse_multiline(injury_widgets['red_zones'].value),
            available_equipment=parse_multiline(availability_widgets['available_equipment'].value or availability_widgets['logistics'].value),
            strength_routines=parse_multiline(injury_widgets['strength_routines'].value),
            impact_limitations=parse_multiline(injury_widgets['impact_limitations'].value),
            feedback_required=psych_widgets['feedback_required'].value,
        )

        # Race times and VDOT
        recent_dist_label = physiology_widgets['recent_distance'].value
        recent_time_str = physiology_widgets['recent_time'].value.strip()
        zones = None
        if recent_time_str:
            km = distance_to_km(recent_dist_label)
            if km:
                try:
                    zones = TrainingZones(method='jack_daniels')
                    zones.add_race_time('recent', RaceTime.from_time_string(km, recent_time_str))
                    zones.calculate_zones()
                    profile.recent_race_times[recent_dist_label] = recent_time_str
                    profile.vdot_estimate = zones.vdot
                except Exception as exc:
                    print('Não foi possível calcular VDOT:', exc)

        # Plano exemplo
        plan = PlanGenerator.generate_plan(
            name=f"Plano {main_goal.distance}",
            goal=main_goal.distance,
            level=history_widgets['experience_level'].value,
            days_per_week=availability_widgets['days_per_week'].value,
            training_zones=zones,
            user_profile=profile,
        )

        # Saídas
        print('📊 Perfil consolidado:')
        pprint(profile.to_dict(), sort_dicts=False)

        print()  # linha em branco
        print('🏃 Plano (resumo JSON):')
        plan_dict = plan.to_dict()
        print(json.dumps(plan_dict, indent=2))

        print()  # linha em branco
        print('💾 Dicas: salve o perfil para reuso com json.dump(profile.to_dict()) e regenere planos ajustando VDOT/testes.')

save_button.on_click(build_profile)

display(save_button, output)
